In [34]:
#Cleaning the data before other analysis.
#adding all dates in the range of the input file for all assets
#add row indexes after this cleaning, to make them unique

import pandas as pd
import datetime
from datetime import datetime, timedelta

data = pd.read_csv("/Users/aselab/Qsync/VizWorx/SampleData/TGI Breadcrumb Grid Dec 10th-Jan 16th.csv")

#covert string to date time
times = pd.to_datetime(data['Breadcrumb Date UTC'], format= '%Y-%m-%d %H:%M:%S')
#append the times as utc_time column to the dataframe
data.loc[:, "utc_time"] = pd.Series(times , index = data.index)

df = data.loc[:,['AssetID','utc_time', 'City', 'Province','Country','Latitude','Longitude']]


#Cleaning the data
#first: finding out the date range in the input file
def range_date(date_list):
    return(min(date_list) , max(date_list))


range_dates = range_date(list(df['utc_time']))

#returns list of dates that should be added to each asset (missing dates)
#inputs: dataframe and a string that shows the daily time (7:59), time range of the input file
def list_of_dates_to_be_added (asset_data_frame, daily_hour, min_time_range_input_file, max_time_range_input_file):     
    #converting the daily_hour to datetime object
    morning_time = datetime.strptime(daily_hour, '%H:%M').time()
    range_dates_max_date = datetime.fromtimestamp(max_time_range_input_file.timestamp()).date()
    range_dates_max_morning_date = datetime.combine(range_dates_max_date, morning_time)
    #changing the times of the input asset dataframe to date type
    time_list = []
    for item in list(asset_data_frame['utc_time']):
        time_list.append(datetime.fromtimestamp(item.timestamp()).date())

    #making a uniques list of the time_list
    unique_time_list = list(set(time_list))
    #creating list of dates in the time range of the input file
    dates_list = []
    for i in range(1, (max_time_range_input_file - min_time_range_input_file).days + 1):
        dates_list.append( (range_dates_max_morning_date - timedelta(days = i)).date())
    #finding out the dates that are present it time range of the input file but not in the input asset dataframe
    new_list_dates = [item for item in dates_list if item not in unique_time_list]
    new_list_datetimes=[]
    for item in new_list_dates:
        new_list_datetimes.append(datetime.combine(item, morning_time))
    
    return new_list_datetimes

#max_row_index = my_df['row_index'].max()

data_frame_last_index = df.index[-1]

def insert_row(data_frame, list_of_dates):
    last_row = len(data_frame) - 1
    list_of_dict = []
    d = {}
    #set this to turn off the warning error
    data_frame.is_copy = False
    for i in range(0, len(list_of_dates)):
        data_frame.loc[i + last_row] = [data_frame.iloc[last_row]['AssetID'], list_of_dates[i], data_frame.iloc[last_row]['City'], data_frame.iloc[last_row]['Province'],data_frame.iloc[last_row]['Country'], data_frame.iloc[last_row]['Latitude'], data_frame.iloc[last_row]['Longitude']] 
    return data_frame

grouped_df = df.groupby('AssetID')
my_df = df[df['AssetID'] == 'CGEX1797']
input_dates = list_of_dates_to_be_added(my_df, "7:59", range_dates[0], range_dates[1])
insert_row(my_df, input_dates)
my_df

my_df_2 = df[df['AssetID'] == 'CGEX1348']
input_dates = list_of_dates_to_be_added(my_df_2, "7:59", range_dates[0], range_dates[1])


#make a list of dataframes
appended_data = []
for name, group in grouped_df:
    n_df = insert_row(grouped_df.get_group(name), input_dates)
    appended_data.append(n_df)

#Make a dataframe out of the list
#set ignore_index= True to return unique values
result = pd.concat(appended_data, ignore_index =True)
numbers = len(result)
new_index = pd.Series(list(range(0,numbers)))
result.loc[:,'new_index'] = new_index
result.to_csv("/Users/aselab/Qsync/VizWorx/test_4.csv")
result.sort_values(by = ['new_index'])
#type(result)
#result.tail(10)

,AssetID,utc_time,City,Province,Country,Latitude,Longitude,new_index
0,CGAX9054,2018-01-16 06:38:22,Port Coquitlam,BC,CA,49.251816,-122.744431,0
1,CGAX9054,2018-01-16 05:49:40,Port Coquitlam,BC,CA,49.251858,-122.744362,1
2,CGAX9054,2018-01-16 05:08:29,Port Coquitlam,BC,CA,49.251839,-122.744301,2
3,CGAX9054,2018-01-16 04:39:07,Port Coquitlam,BC,CA,49.251904,-122.744431,3
4,CGAX9054,2018-01-16 04:02:45,Port Coquitlam,BC,CA,49.251858,-122.744469,4
5,CGAX9054,2018-01-16 03:25:43,Port Coquitlam,BC,CA,49.251923,-122.744217,5
6,CGAX9054,2018-01-16 02:42:07,Port Coquitlam,BC,CA,49.251846,-122.744255,6
7,CGAX9054,2018-01-16 01:56:42,Port Coquitlam,BC,CA,49.251804,-122.744255,7
8,CGAX9054,2018-01-16 01:38:32,Port Coquitlam,BC,CA,49.252129,-122.744408,8
9,CGAX9054,2018-01-16 00:56:25,Port Coquitlam,BC,CA,49.252029,-122.744278,9
